<a href="https://colab.research.google.com/github/sabyadg/Clustering1D-Data/blob/master/1D-Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

In [0]:
import numpy as np
import pandas as pd

In [0]:
files=["2018-10-15_11-30-03-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-31-41-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-33-03-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-36-36-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-38-01-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-39-18-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-40-25-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-44-00-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-45-03-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-48-43-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-53-51-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-55-18-PASSTHROUGH-12-59-31.npy",
"2018-10-15_11-56-44-PASSTHROUGH-12-59-31.npy",
"2018-10-15_12-02-56-PASSTHROUGH-12-59-31.npy",
"2018-10-15_12-04-10-PASSTHROUGH-12-59-31.npy",
"2018-10-15_12-05-27-PASSTHROUGH-12-59-31.npy",
"2018-10-18_13-58-38-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-00-47-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-02-47-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-05-15-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-06-45-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-07-48-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-10-19-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-11-51-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-13-11-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-18-15-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-19-14-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-20-19-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-24-33-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-25-40-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-26-39-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-29-51-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-30-51-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-32-01-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-35-02-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-36-01-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-38-29-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-54-45-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-55-53-PASSTHROUGH-12-59-31.npy",
"2018-10-18_14-56-57-PASSTHROUGH-12-59-31.npy"]
A = []
for fname in files:
    inf_from_every_file = np.load(fname)
    A.append(inf_from_every_file)
A = np.concatenate(A)

In [0]:
print(' No. of files' , len(files),'\n' , 'No. of points in each file', A.shape[0]/len(files),'\n' ,'Total no. of points', A.shape[0])

In [0]:
raw_df = pd.DataFrame()
raw_df['data']  = A
#raw_df.head()
raw_df.describe()

In [0]:
import seaborn as sns
 
#sns.distplot(raw_df['data'],kde=False, color = 'r')
raw_df['normalized'] =   (raw_df['data'] - raw_df['data'].mean())/raw_df['data'].std()
 
sns.distplot(raw_df['normalized'] , kde=False, color='b')

In [0]:
raw_df.quantile([0.01,0.05,0.95, 0.99])

In [0]:
#del(raw_df_5_95q)
raw_df_5_95q = raw_df.query('normalized > -0.289080  & normalized <0.31')
print('No. of Points to model', raw_df_5_95q.shape[0])
sns.distplot(raw_df_5_95q['normalized'],kde=False,color='r')

In [0]:
raw_df_L = raw_df.query('normalized < -0.289080')
print('No. of points in the Left axis, which will not be modelled are ',raw_df_L.shape[0])
sns.distplot(raw_df_L['data'],kde=False,color='b')
raw_df_R = raw_df.query('normalized > 0.31')
print('No. of points in the Right axis, which will not be modelled are ', raw_df_R.shape[0])
sns.distplot(raw_df_R['data'],kde=False,color='g')

In [0]:
clean_df = raw_df_5_95q.copy(deep=True)
clean_df.head()

In [0]:
from sklearn.cluster import KMeans as km
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
X = np.array(clean_df['data']).reshape(-1, 1)
X = StandardScaler().fit_transform(X)
sns.distplot(X,kde=False)
X.shape[0]

In [0]:
import time
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

# #############################################################################
# Compute clustering with Means

k_means = KMeans(init='k-means++', n_clusters=3, n_init=10)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0

# #############################################################################
# Compute clustering with MiniBatchKMeans
batch_size=X.shape[0]
mbk = MiniBatchKMeans(init='k-means++', n_clusters=3, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)
t0 = time.time()
mbk.fit(X)
t_mini_batch = time.time() - t0

In [0]:
fig = plt.figure(figsize=(8, 3))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
colors = ['#4EACC5', '#FF9C34', '#4E9A06']
n_clusters = 3
# We want to have the same colors for the same cluster from the
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# closest one.
k_means_cluster_centers = np.sort(k_means.cluster_centers_, axis=0)
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis=0)
k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)
order = pairwise_distances_argmin(k_means_cluster_centers,
                                  mbk_means_cluster_centers)

# KMeans
ax = fig.add_subplot(1, 3, 1)
for k, col in zip(range(n_clusters), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], 'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0],   'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())
#plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (
#    t_batch, k_means.inertia_))
#
# MiniBatchKMeans
ax = fig.add_subplot(1, 3, 2)
for k, col in zip(range(n_clusters), colors):
    my_members = mbk_means_labels == order[k]
    cluster_center = mbk_means_cluster_centers[order[k]]
    ax.plot(X[my_members, 0],  'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0],   'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('MiniBatchKMeans')
ax.set_xticks(())
ax.set_yticks(())
#plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' %
#         (t_mini_batch, mbk.inertia_))

# Initialise the different array to all False
different = (mbk_means_labels == 4)
ax = fig.add_subplot(1, 3, 3)

for k in range(n_clusters):
    different += ((k_means_labels == k) != (mbk_means_labels == order[k]))

identic = np.logical_not(different)
ax.plot(X[identic, 0], 'w',
        markerfacecolor='#bbdfbb', marker='.')
ax.plot(X[different, 0], 'w',
        markerfacecolor='m', marker='.')
ax.set_title('Difference')
ax.set_xticks(())
ax.set_yticks(())

plt.show()

In [0]:
clean_df['kmeans-labels'] =  k_means_labels
clean_df['mini-batch-kmeans-labels'] =  mbk_means_labels

In [0]:
clean_df.groupby('kmeans-labels').data.count()
sns.distplot(clean_df['kmeans-labels'],kde=False)

#clean_df.groupby('mini-batch-kmeans-labels').data.count()
#sns.distplot(clean_df['mini-batch-kmeans-labels'],kde=False)


In [0]:
sns.distplot(clean_df[clean_df['kmeans-labels'] == 0]['normalized'], color = 'r')
sns.distplot(clean_df[clean_df['kmeans-labels'] == 1]['normalized'], color = 'b')
sns.distplot(clean_df[clean_df['kmeans-labels'] == 2]['normalized'], color = 'g')

In [0]:
sns.distplot(clean_df[clean_df['kmeans-labels'] == 0]['data'], color = 'r')
sns.distplot(clean_df[clean_df['kmeans-labels'] == 1]['data'], color = 'b')
sns.distplot(clean_df[clean_df['kmeans-labels'] == 2]['data'], color = 'g')